# WNC Library Functionality Testing

In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [2]:
examples = [
    "Sir Alex Ferguson is the greatest football manager of all time.",
    "Sir Alex Ferguson is a great football manager.",
    "Sir Alex Ferguson is a football manager.",
    "the news-press was sold by the new york times company in 2000, and is now independently owned by wendy p. mccaw, a local resident and a biased and eerily thick-headed woman .",
    "it also marked the last season in quarteback 's brett favre illustrious career as a packer .",
]

## `SubjectivityNeutralizer` Testing

In [3]:
from src.inference import SubjectivityNeutralizer

In [4]:
MODEL_PATH = "/home/cdsw/models/bart-tst-full"
sn = SubjectivityNeutralizer(model_identifier=MODEL_PATH)

In [5]:
sn.transfer(examples)

['Sir Alex Ferguson is one of the greatest football managers of all time.',
 'Sir Alex Ferguson is a football manager.',
 'Sir Alex Ferguson is a football manager.',
 'the news-press was sold by the new york times company in 2000, and is now independently owned by wendy p. mccaw, a local resident.',
 "it also marked the last season in quarteback 's career."]

In [6]:
tests = [
    "Sir Alex Ferguson is the greatest football manager of all time.",
    "Sir Alex Ferguson is considered by some to be the greatest football manager of all time.",
    "Sir Alex Ferguson is one of the greatest football managers of all time.",
    "Sir Alex Ferguson is widely regarded as one of the greatest football managers of all time",
]

## `StyleIntensityClassifier` Testing

In [7]:
from src.inference import StyleIntensityClassifier

In [8]:
MODEL_PATH = "../models/bert-cls-full3/checkpoint-96000"
sc = StyleIntensityClassifier(model_identifier=MODEL_PATH)

In [9]:
tests

['Sir Alex Ferguson is the greatest football manager of all time.',
 'Sir Alex Ferguson is considered by some to be the greatest football manager of all time.',
 'Sir Alex Ferguson is one of the greatest football managers of all time.',
 'Sir Alex Ferguson is widely regarded as one of the greatest football managers of all time']

In [10]:
sc.score(tests)

[{'label': 'LABEL_0',
  'score': 0.9861143827438354,
  'distribution': [0.9861143827438354, 0.013885647989809513]},
 {'label': 'LABEL_1',
  'score': 0.833655059337616,
  'distribution': [0.16634498536586761, 0.833655059337616]},
 {'label': 'LABEL_0',
  'score': 0.9462658762931824,
  'distribution': [0.9462658762931824, 0.05373416841030121]},
 {'label': 'LABEL_1',
  'score': 0.8195207118988037,
  'distribution': [0.18047933280467987, 0.8195207118988037]}]

In [11]:
sc.score(examples)

[{'label': 'LABEL_0',
  'score': 0.9861143827438354,
  'distribution': [0.9861143827438354, 0.013885647989809513]},
 {'label': 'LABEL_0',
  'score': 0.9860873818397522,
  'distribution': [0.9860873818397522, 0.013912606984376907]},
 {'label': 'LABEL_1',
  'score': 0.9913819432258606,
  'distribution': [0.008618118241429329, 0.9913819432258606]},
 {'label': 'LABEL_0',
  'score': 0.9853444695472717,
  'distribution': [0.9853444695472717, 0.014655554667115211]},
 {'label': 'LABEL_0',
  'score': 0.9856260418891907,
  'distribution': [0.9856260418891907, 0.014373908750712872]}]

### Calculate Style Transfer Intensity

In [12]:
# transfer style from examples
input_text = examples[:2]
output_text = sn.transfer(examples[:2])

text_pairs = {"input_text": input_text, "output_text": output_text}
text_pairs

{'input_text': ['Sir Alex Ferguson is the greatest football manager of all time.',
  'Sir Alex Ferguson is a great football manager.'],
 'output_text': ['Sir Alex Ferguson is one of the greatest football managers of all time.',
  'Sir Alex Ferguson is a football manager.']}

In [13]:
sc.calculate_transfer_intensity(**text_pairs)

[0.0398, 0.9775]